In [ ]:
'''RAFT Polymerisation Simulation
by Sophie Kay, Anne-Sophie Korotov, Anastasiia Lapina, Amelie Mattheus'''

In [ ]:
import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import random
%matplotlib inline

def averagePolymerLength(data):
    '''calculates average polymer length from data files 
     input: data stored as 2D array with column 1 being polymer length and column 2 being the probability
     output: average degree of polymerisation
     '''
        
    polymerLengths = data[0]
    polymerProb = data[1]

    polymerAverage = np.sum(polymerLengths * polymerProb)

    return polymerAverage

def calculateWidth(data):
    ''' calculate polydispersity of polymers from data 2D array in files
    input: average degree of polymerisation from averagePolymerLength function
    output: with of distribution (polydispersity) ''' 
    avgPLen = averagePolymerLength(data)

    w = np.sum(np.multiply(np.power(data[0] - avgPLen, 2), data[1]))

    return w

def getDataFromFile(file):
    ''' extracts data from text files  
    input: text files with snapshot of system from every 10s 
    output: data from text files'''
    return np.loadtxt(file, delimiter=",")
    
def saveDataToFile(file, data):
    ''' saves data to file
    input: data as 2D array 
    gives: file with data '''
    np.savetxt(file, data, delimiter=",")

def plotResults(files):
    '''plots graph of polymer length over probability 
    input: data as 2D array extracted with getDataFromFile function from files 
    shows:  graph with x-axis: polymer length (data[0]), y-axis: probability (data[1])
    '''
    
    for file in files:
        data = getDataFromFile(file)
        plt.scatter(data[0], data[1])
        plt.title("Degree of Polymerisation")
        plt.xlabel("Degree of Polymerisation")
        plt.ylabel("Probability")
        plt.savefig(file[:-4] + ".png")
        plt.show()

In [ ]:



NA = 6.022E23 # Avagadro's constant


class updateTime(): 
    '''class that updates the time'''
    def __init__(self, time):
        
        self.time=time
        self.saveIter = 0 # number of iterationsS
    
    def tauPassed(self, Rtot):
        '''calculates tau
        input: total rates, random number from one to zero
        output: time and tau passed  '''
        r=np.random.uniform(0,1) #chooses random number r with uniform probability
        tau=(1/Rtot)*np.log(1/r)# calculates tau
        
        if self.time % 10 > (self.time+tau) % 10: # checks if ten seconds passed 
            self.time+=tau
            self.saveIter += 1
            return True
        else: 
            self.time+=tau
            return False

class initialConditions(object):
    
    def __init__(self, cRAFT, cInitiator, cMonomer, k31, k32, k51, vol = 0.01, cRadical = 0, cPolymer = 0, cAdduct = 0, cRAFTradical = 0, cRm = 0, rates = []):
        # specifies conditions and kinetic constants
        self.k32 = k32
        self.k31 = k31
        self.k51 = k51
        
        self.rates = rates 
        self.cRAFT = cRAFT
        self.vol = vol
        self.nRAFT = (self.cRAFT*NA*self.vol)

        self.cInitiator = cInitiator
    
        self.nInitiator = (self.cInitiator*NA*self.vol)
        self.cMonomer = cMonomer
        self.nMonomer = (self.cMonomer*NA*self.vol)
        
        self.cRadical = cRadical
        self.nRadical = (self.cRadical*NA*self.vol)
        self.cPolymer = cPolymer
        self.nPolymer = (self.cPolymer*NA*self.vol)
        self.cAdduct = cAdduct 
        self.nAdduct = (self.cAdduct*NA*self.vol)
        self.cRAFTradical = cRAFTradical # Polymer with RAFT agent attached as radical 
        self.nRAFTradical = (self.cRAFTradical*NA*self.vol)
        self.cfinalpolymer = 0 # non-living polymer molecule concentration
        self.nfinalpolymer = (self.cfinalpolymer*NA*self.vol)
        self.cDeadTRn = 0 # polymer with RAFT agent attached but not a radical 
        self.nDeadTRn = 0
  
        self.rates = rates
        
        self.nRn = np.zeros((10000), dtype=int) # stores length of living polymers
        self.nDotTRn = np.zeros((10000), dtype=int) # stores length of polymer radical with RAFT agent attached 
        self.nRnTRm = np.zeros((10000,10000), dtype=int) # stores lengths defining adduct (m and n)
        self.nPn = np.zeros((10000), dtype=int) # stores length of dead polymers
        self.nTRn = np.zeros((10000), dtype=int) # stores length of polymer chain with RAFT agent attached 
        return


class ReactionRates(initialConditions):
    '''calculates Reaction rates using concentrations given '''
    
    def __init__(self, cRAFT, cInitiator, cMonomer, k31, k32, k51,  vol = 0.000001, cRadical = 0, cPolymer = 0, cAdduct = 0, cRAFTradical = 0, rates = []):
        super().__init__(cRAFT, cInitiator, cMonomer, k31, k32, k51)
    
    def RxInitiation(self, rates):
        '''calculates initiation rate 
        input: kI, concentration of initiator
        output: Rx (reaction rate for initiation) appended to list of rates with tags for identification of the rates later
        ''' 
        kI = 0.36
        Rx = kI*self.cInitiator/(2)
        rates.append(("Rx",Rx))
        
        return rates
    
    def RxPropagation(self, rates): 
        '''calculates propagation rate
        input: kP, concentration of living polymers (cPolymers), concentration of radicals produced by initiation (cRadicals)
        output: RxP1, RxP2, RxP3 (reaction rates of the three possible reactions that can happen during propagation) with tags in rates list 
        '''
        kP = 3.6E7
        if self.cRadical + self.cPolymer == 0: # if no polymer (dimers) present yet
            RxP1 = 0
            RxP2 = 0
        else: 
            RxP1 = kP*self.cMonomer*self.cRadical
            RxP2 = kP*self.cMonomer*self.cPolymer 
            
        rates.append(("RxP1",RxP1)) # puts all rates into list with tags 
        rates.append(("RxP2",RxP2))
        return rates
     
    def RxPreEqu(self, rates):
        '''calculates rates for pre-equilibrium
        input: k31, k32, concentration of RAFT radicals (cRAFTradical, denoted in pdf as DotTRn), concentration of living polymers (cPolymers), concentration of RAFT agent (cRAFT, T in pdf)
        output:Rx31, Rx32, Rx33 (reaction rates of the three possible reactions that can happen during the pre-equilibrium) with tags in rates list '''
        k31 = self.k31
        k32 = self.k32
        k33 = self.k32
        
        Rx31 = k31*self.cRAFT*self.cPolymer
        Rx32 = k32*self.cAdduct
        Rx33 = k33*self.cAdduct 
        
        rates.append(("Rx31",Rx31))
        rates.append(("Rx32",Rx32))
        rates.append(("Rx33",Rx33))
        return rates
    
    def RxCoreEqu(self, rates): 
        '''calculates rates for core equilibrium
        input: k31, k32, concentration of RAFT radicals (cRAFTradical, denoted in pdf as DotTRn), concentration of living polymers (cPolymers), concentration of Adduct (cAdduct)
        output: reaction rates of reaction 4.1, 4.2, 4.3 (Rx41, Rx42, Rx42) in rates list with tags 
        '''
        k41 = self.k31 
        k42 = self.k32
        k43 = self.k32

        Rx41 = k41 * self.cPolymer * self.cDeadTRn 
        Rx42 = k42 * self.cAdduct 
        Rx43 = k43 * self.cAdduct 
        rates.append(("Rx41",Rx41))
        rates.append(("Rx42",Rx42))
        rates.append(("Rx43",Rx43))
        return rates
    
    def RxTermination(self, rates):
        '''calculates termination rates
        input: k51, concentration of living polymers (cPolymers), concentration of Adduct (cAdduct), concentration of Radicals with no monomer attached (cRadical, denoted as R0 in pdf)
        output: reaction rates for reaction 5.1, 5.2, 5.3, 5.4 (Rx51, Rx52, Rx53, Rx54) in rates list with tags'''
        k51 = self.k51
        k52 = self.k51
        k53 = self.k51
        k54 = self.k51
        
        Rx51 = k51 * self.cPolymer * self.cPolymer
        Rx52 = k52 * self.cPolymer * self.cPolymer
        Rx53 = k53 * self.cAdduct * self.cRadical
        Rx54 = k54 * self.cAdduct * self.cPolymer
        
        rates.append(("Rx51",Rx51))
        rates.append(("Rx52",Rx52))
        rates.append(("Rx53",Rx53))
        rates.append(("Rx54",Rx54))
        
        return rates
    
    def getAllRates(self):
        '''runs all above functions in order to get all the reaction rates
        input: rates calculated above
        output: list of all rates of all possible reactions with tags for identification'''
        rates = []
        rates = self.RxInitiation(rates)
        rates = self.RxPropagation(rates)
        rates = self.RxPreEqu(rates)
        rates = self.RxCoreEqu(rates)
        rates = self.RxTermination(rates)
        
        return rates
        
        
    
        
class Reaction(ReactionRates): 
    '''updates number of molecules after each reaction'''
    
    def __init__(self, cRAFT, cInitiator, cMonomer, k31, k32, k51, vol = 0.00001, cRadical = 0, cPolymer = 0, cAdduct = 0, cRAFTradical = 0):
        super().__init__(cRAFT, cInitiator, cMonomer, k31, k32, k51)
        
        self.timeCalculator = updateTime(0) #sets time 
     
        
    def chooseRn(self):
        '''choose living polymer of length n with certain probability
        input: 1D array of living polymers with index indicating n (degree of polymerisation) and the number stored at that index being the number of living polymers of that length  
        output: index (i.e. living polymer with length n) chosen to react 
        ''' 
        numberOfRns = np.sum(self.nRn)
        assert numberOfRns > 0
        probabilityDistribution = self.nRn.copy() / numberOfRns
        
        chosenIndex = np.random.choice(10_000, 1, p=probabilityDistribution)[0]
        
        return chosenIndex
    
    def chooseDotTRn(self):
        ''' choose living  RAFT Radicals (denoted as Dot TRn in pdf) of length n with certain probability
        input: 1D array (called nDotTRn) with the index indicating n and the number stored at that index being the number of living polymers RAFT radicals with that length n 
        output: index chosen (i.e. RAFT radical with certain length n that will react)
        '''
        numberOfDotTRns = np.sum(self.nDotTRn)
        assert numberOfDotTRns > 0 # errror if no Polymers of length n 
        probabilityDistribution = self.nDotTRn.copy() / numberOfDotTRns
        
        chosenIndex = np.random.choice(10_000, 1, p=probabilityDistribution)[0]
        
        return chosenIndex
    
    
    def chooseTRn(self):
        '''choose polymers with RAFT agent attached of length n with certain probability 
        input: 1D array (called nTRn) with the index indicating n and the number stored at that index being the number of polymers with attached RAFT agent with that length n 
        output: index chosen (i.e. polymer with RAFT agent attached with certain length n that will react)
        '''
        numberOfTRns = np.sum(self.nTRn)
        assert numberOfTRns > 0 # errror if no Polymers of length n
        probabilityDistribution = self.nTRn.copy() / numberOfTRns
        
        chosenIndex = np.random.choice(10_000, 1, p=probabilityDistribution)[0]
        
        return chosenIndex
    
    def chooseRnTRm(self):
        '''choose Adduct with lengths n and m
        input: 2D Array with index 1 (chosenNINdex) indicating length n and index 2 (chosenMIndex) indicating length m
        output: chosen indices n and m (i.e. adduct at nRnTRm[chosenNIndex][chosenMIndex] with lengths n and m that will react)
        '''
        numberOfN = np.sum(self.nRnTRm, axis=1)
        probabilityDistributionN = numberOfN / np.sum(numberOfN)
        
        chosenNIndex = np.random.choice(10_000, 1, p=probabilityDistributionN)[0]        
        
        numberOfM = np.sum(self.nRnTRm[chosenNIndex])
        
        assert numberOfM > 0 # errror if no Polymers of length M 
        probabilityDistributionM = self.nRnTRm[chosenNIndex].copy() / numberOfM
        
        chosenMIndex = np.random.choice(10_000, 1, p=probabilityDistributionM)[0]
        
        return chosenNIndex, chosenMIndex
    
    
        
    def getLivingPolymers(self):
        '''calculates total number of living polymers
        input: arrays nRn, nRnTRm
        output: 2D array with all living polymers with first index being the length of the polymers and the second index being the total number of
        living polymers with that length
        '''
        totalLivingPolymers = np.sum(self.nRn) + np.sum(self.nDotTRn)  #sum number of living polymers
            
        totalLivingPolymers += np.sum(np.sum(self.nRnTRm, axis=1)) # sum number of adducts and add that to the sum of living polymers 

        livingPolymersByLength = self.nRn.copy() # adds the length of the adducts to the array containing all polymers 
        for i in range(10_000):
            for j in range(10_000):
                if i + j + 2 < 10_000:
                    livingPolymersByLength[i+j+2] += self.nRnTRm[i,j]
                else:
                    break

        livingPolymersByLength = livingPolymersByLength / max(totalLivingPolymers,1) # calculates probability distribution of certain polymer with length n 
       
        arrangeArray = np.arange(1,10_001)

        
        return np.array([arrangeArray, livingPolymersByLength])
   
    def chooseReaction(self, rates, x = 1):
        '''chooses reaction from rates using probability associated with each reaction and checks if a file needs to be saved and whether simulation ran for an hour
        input: rates list, x to loop and through rates list and get the values for the rates
        output: true if simulation ran for an hour, false if it didn't'''
        ps = np.zeros(len(rates)) #numpy array with zeros of the same length as the rates list, stores the probabilities for each reaction
        
        sumRates = sum([x[1] for x in rates]) # sums all rates
        
        for i in range(len(rates)): 
            ps[i] = rates[i][1]/sumRates       # uses sum of all rates to calculate the probability
        

        newReaction = random.choices(rates, weights=ps)[0] # chooses reaction using probabilities in ps 

  
        if newReaction[0] == "Rx": # compares tags of chosen reaction and calls on the function to update the system using the reaction
            self.Initiation()
        elif newReaction[0] in ["RxP1","RxP2"]:
            self.Propagation(newReaction[0])
        elif newReaction[0] in ["Rx31","Rx32","Rx33"]:
            self.PreEqu(newReaction[0])
        elif newReaction[0] in ["Rx41","Rx42","Rx43"]:
            self.CoreEqu(newReaction[0])
        elif newReaction[0] in ["Rx51","Rx52","Rx53","Rx54"]:
            self.Termination(newReaction[0])
            
        needToSave = self.timeCalculator.tauPassed(sumRates) # checks if file needs to be saved 
        
        if needToSave: # saves file including all living polymers if 10s passed 
            
            fileNumber = self.timeCalculator.saveIter
            filename = 'system_{:06d}.csv'.format(fileNumber) #save file with formated file number as csv
            saveDataToFile(filename, self.getLivingPolymers())
            plotResults(['system_{:06d}.csv'.format(fileNumber)])
            print(filename, "Saved")
            
            if fileNumber == 360: #stops simulation if 1h passed 
                return True 
            
                
        return False 
    
    def Initiation(self):
        """updates the number of molecules and concentrations if the reaction chosen was initiation: reactant: I, product: 2R0
        uses: number of initiator molecules (nInitiator), number of radical (R0) molecules (nRadical), 
        1D array nRn which is updated by adding Radicals, as 2 'polymers'with degree of polymerisation of 0 produced
        calculates: updated numbers of reactants and products, updated nRn array
        """

        self.nInitiator -= 1 # updates nInitiator
        self.nRadical += 2 # updates nRadical
        
        # calculates updated concentrations: 
        self.cRadical = (self.nRadical)/(NA*self.vol)
        self.cInitiator = (self.nInitiator)/(NA*self.vol)

    
    def Propagation(self, reactionType):
        '''updates the number of molecules and concentrations if the reaction chosen was propagation:  # reactant: R0, M, product: Polymer (Rn+1)
        uses: chosen polymer with length n from chooseRn, concentration and number of Monomers (cMonomer, nMonomer), 
        concentration and number of living Polymers (cPolymer, nPolymer)
        calculates: updated array nRn (updated degree of polymerisation of polymer), updated nMonomer, cMonomer, cPolymer, nPolymer '''
        #for reaction of monomer with radical R0 
        if reactionType == "RxP1":
            # update number of monomers and polymers
            self.nMonomer -= 1 
            self.nRadical -= 1
            self.nPolymer += 1
            
            self.nRn[0] += 1 #updates degree of polymerisation
        # for reaction of monomer with polymer radical of length n 
        elif reactionType == "RxP2":
            # update number of monomers
            self.nMonomer -= 1
            x = self.chooseRn() # choose polymer with degree of polymerisation n
            if x+1 < 10_000:
                # degree of polymerisation update from n to n + 1  
                if self.nRn[x] != 0:
                    self.nRn[x] -= 1
                    self.nRn[x+1] += 1
            
        # convert back to concentration 
        self.cMonomer = (self.nMonomer)/(NA*self.vol) 
        self.cPolymer = (self.nPolymer)/(NA*self.vol)
        self.cRadical = (self.nRadical)/(NA*self.vol)
        

    def PreEqu(self, reactionType):   
        '''updates the number of molecules and concentrations if the reaction chosen was a pre-equilibrium reaction
        uses: reaction type, number of living polymer molecule (nPolymer), RAFT agents (nRAFT), adduct molecules (nAdduct),
        array nRn of polymer molecules with length n, array nDotTRn of RAFT radical molecules with polymer length n 
        calculates: updated number of molecules for reactants and producst, updated arrays nRn and nDotTRn'''
        
        # for 3.1 : reactant: living Polymer, RAFT agent, product: radical with RAFT agent attached (TRn Radical)
        if reactionType == "Rx31": #update number of molecules if reaction chosen reaction 3.1
            self.nPolymer -= 1
            self.nRAFT -= 1
            self.nRAFTradical += 1 

            
            x = self.chooseRn() # choose reacting polymer with length n 
            if self.nRn[x] != 0: # update nRn and nDotTRn 
                self.nRn[x] -= 1
                self.nDotTRn[x] += 1 
            
            #calculate updated concentrations:
            self.cPolymer =  (self.nPolymer)/(NA*self.vol)
            self.cRAFT =  (self.nRAFT)/(NA*self.vol)
            self.cRAFTradical =  (self.nRAFTradical)/(NA*self.vol) 
        # for 3.2 reactants: RAFT radical with polymer attached, products: living polymer, RAFT agent
        elif reactionType == "Rx32": #update number of molecules if reaction chosen reaction 3.2
            self.nRAFTradical -= 1 
            self.nPolymer += 1
            self.nRAFT += 1
            
            x = self.chooseDotTRn() # choose RAFT radical of length n
            if self.nDotTRn[x] != 0: # update nRn and nDotTRn 
                self.nDotTRn[x] -= 1
                self.nRn[x] += 1 
                
            #calculate updated concentration: 
            self.cRAFTradical =  (self.nRAFTradical)/(NA*self.vol) 
            self.cPolymer =  (self.nPolymer)/(NA*self.vol)
            self.cRAFT =  (self.nRAFT)/(NA*self.vol)
       
    
        #for 3.3: reactants: RAFT Radical with polymer attached, products: radical (R0), polymer with raft agent attached but not a radical
        else: 
            
            self.nRAFTradical -= 1 #update number of RAFT radicals
            self.nDeadTRn += 1
            
            x = self.chooseDotTRn() 
            if self.nDotTRn[x] != 0: #update degree of polymerisation
                self.nDotTRn[x] -= 1
                self.nTRn[x] += 1
            
            # update concentration: 
            self.cRAFTradical = (self.nRAFTradical)/(NA*self.vol)
            self.cDeadTRn = (self.nDeadTRn)/(NA*self.vol)
#             self.cRAFT =  (self.nRAFT)/(NA*self.vol)
           
    def CoreEqu(self, reactionType):
        '''updates the number of molecules and concentrations if the reaction chosen was a core equilibrium reaction
        uses: reaction type, chosen molecule from array nTRn and nRn, number of living polymers (nPolymer, cPolymer), 
        RAFT radicals (nRAFTradical), adducts (nAdduct),
        calculates: updated number and concentrations of reactants and updated degree of polymerisations of reacting polymers, RAFT radicals with polymers
        and Adducts'''
        #4.1 reactant: living polymer, RAFT radical with polymer attached , product: Adduct
        if reactionType == "Rx41": 
            xn = self.chooseRn() #choose reacting polymers with degree of polymerisatons from arrays nRn and nTRn
            xm = self.chooseTRn()
           
        #updating number of molecules
            self.nPolymer -= 1 
            self.nRAFTradical -=1 
            self.nAdduct +=1 
        
         #updating degree of polymerisation   
            if self.nRn[xn] != 0 and self.nTRn[xm] != 0:
                self.nRn[xn] -= 1
                self.nTRn[xm] -= 1
                self.nRnTRm[xn,xm] += 1
                
                
            #calculate updated concentrations
            self.cPolymer =  (self.nPolymer)/(NA*self.vol)
            self.cRAFT =  (self.nRAFT)/(NA*self.vol)      
       
        #4.2 reactants: Adduct, products: living polymer, RAFT radical with polymer attached 
        elif reactionType == "Rx42": 
             #updating number of molecules:
            self.nPolymer += 1
            self.nDeadTRn += 1
            self.nAdduct -= 1 
            
            xn, xm = self.chooseRnTRm() # choose adducts with certain degree of polymerisation n and m 
            if self.nRnTRm[xn,xm] != 0: #update degree of polymerisation of adducts, living polymers, and RAFT radical with monomers attached 
                self.nRnTRm[xn,xm] -= 1
                self.nTRn[xm] += 1
                self.nRn[xn] += 1
           
            #calculate updated concentrations
            self.cPolymer = (self.nPolymer)/(NA*self.vol)
            self.cDeadTRn =  (self.nDeadTRn)/(NA*self.vol)
            self.cAdduct = (self.nAdduct)/(NA*self.vol)


    
        else: #4.3 reactant: Adduct, product: living polymer, RAFT radical with polymer attached 
            # updating number of molecules:
            self.nPolymer += 1
            self.nDeadTRn += 1
            self.nAdduct -= 1 
            
            xn, xm = self.chooseRnTRm() # choose adducts with certain degree of polymerisation n and m 

            if self.nRnTRm[xn,xm] != 0: #update degree of polymerisation of adducts, RAFT radicals with polymer attached, living polymer radicals
                self.nRnTRm[xn,xm] -= 1
                self.nTRn[xn] += 1
                self.nRn[xm] += 1
                
             #calculate updated concentrations
            self.cPolymer = (self.nPolymer)/(NA*self.vol)
            self.cDeadTRn =  (self.nDeadTRn)/(NA*self.vol)
            self.cAdduct = (self.nAdduct)/(NA*self.vol)

    
    def Termination(self, reactionType): 
        '''updates the number of molecules and concentrations if the reaction chosen was a termination reaction
        uses: reaction type, living polymers of length n and m, number of living polymer molecules (nPolymer), number of dead polymer molecules (nfinalpoymer)
        adduct with lengths n and m, number of adducts (nAdduct), number of Radicals (nRadicals) depending on reaction
        calculates: updated number of products and reactants, updated degree of polymerisation of dead polymers , living polymers, adducts in arrays
        '''
        if reactionType == "Rx51":  # 5.1 reactants:living polymers, products: final polymer 
            xn = self.chooseRn() # choose lengths of polymer molecules 
            xm = self.chooseRn()
            if xn + xm < 10_000: # update number of living and dead polymer molecules
                self.nPolymer -= 2 
                self.nfinalpolymer += 1

                if self.nRn[xn] != 0 and self.nRn[xm] != 0: # update degree of polymerisation of living and dead polymer molecules
                    self.nRn[xn] -= 1
                    self.nRn[xm] -= 1
                    self.nPn[xn+xm] += 1
                    
                #calculate updated concentrations: 
                self.cPolymer = (self.nPolymer)/(NA*self.vol)
                self.cfinalpolymer = (self.nfinalpolymer)/(NA*self.vol)
#             
        
        elif reactionType == "Rx52": # 5.2 reactants: living polymers, products: 2 final polymers
            #update number of molecules: 
            self.nPolymer -= 2 
            self.nfinalpolymer +=  2 
            
            xn = self.chooseRn() # choose polymers with degree of polymerisation n and m 
            xm = self.chooseRn()
            if self.nRn[xn] != 0 and self.nRn[xm] != 0: # update degree of polymerisation of living and dead polymer molecules
                self.nRn[xn] -= 1
                self.nRn[xm] -= 1
    
                self.nPn[xn] += 1
                self.nPn[xm] += 1
           
            #calculate updated concentrations: 
            self.cPolymer = (self.nPolymer)/(NA*self.vol)
            self.cfinalpolymer = (self.nfinalpolymer)/(NA*self.vol)
            
# 
            

        elif reactionType == "Rx53": # 5.3 reactants: Adduct, Radical, products: final polymer
            xn, xm = self.chooseRnTRm() #choose adduct molecule with degrees of polymerisation n and m 
            if xn + xm < 10_000: #update number of molecules 
                self.nAdduct -= 1 
                self.nRadical -= 1 
                self.nfinalpolymer += 1 

                if self.nRnTRm[xn, xm] != 0: # update number of adduct molecules with degree of polymerisation n and m 
                    self.nRnTRm[xn, xm] -= 1    

                    self.nPn[xm+xn] += 1 # update number of dead polymer molecules with degree of polymerisation n + m 
                
                 #calculate updated concentrations: 
                self.cAdduct =  (self.nAdduct)/(NA*self.vol)
                self.cfinalpolymer = (self.nfinalpolymer)/(NA*self.vol)
                self.cRadical = (self.nRadical)/(NA*self.vol)
        
        else: # 5.4 reactants: Adduct, living polymer with degree of polymerisation s  , products: dead polymer
            xn, xm = self.chooseRnTRm() #choose adduct
            xs = self.chooseRn() #choose polymer with degree of polymerisation s 
            if xn + xm + xs < 10_000: #update number of molecules
                self.nAdduct -=  1
                self.nPolymer -= 1 
                self.nfinalpolymer += 1

                if self.nRnTRm[xn, xm] != 0 and nRn[xs] != 0: #update number of adduct molecules with degree of polymerisation n and m 
                    self.nRnTRm[xn, xm] -= 1
                    self.nRn[xs] -= 1 #update number of polymer molecules with degree of polymerisation s 

                    self.nPn[xm+xn+xs] += 1 #update number of dead poymer molecules with degree of polymerisation n + m + s

                #calculate updated concentrations: 
                self.cAdduct = (self.nAdduct)/(NA*self.vol)
                self.cfinalpolymer = (self.nfinalpolymer)/(NA*self.vol)
                self.cPolymer = (self.nPolymer)/(NA*self.vol)
    
    


In [ ]:

def runSimulation(conditions): 
    '''runs simulation with certain input conditions
    input: cRAFT, cInitiator, cMonomer, k31, k32, k51 (input in dictionary)
    output: runs simulation and plots width of distribution of degree of polymerisation'''
    reaction = Reaction(conditions["cRAFT"], conditions["cInitiator"], conditions["cMonomer"],conditions["k31"],conditions["k32"],conditions["k51"])
    done = False
    while not done: # runs simulation for 1 h 
        done = reaction.chooseReaction(reaction.getAllRates()) # calls function that checks if 1h has passed 
    
    ws = []
    for i in range(1,360+1): # loops through system for 1h and appends calculated width of distribution from data to list used to then plot distribution
        data = getDataFromFile('system_{:06d}.csv'.format(i))
        
        ws.append(calculateWidth(data)) 
        
        
    plt.plot(np.arange(1,360+1),ws) # plots width of distribution
    plt.xlabel("Time (Seconds)")
    plt.ylabel("Polydispersity")
    plt.title("Polydispersity")
    plt.savefig("PolydispersityGraph.png") # saves figure 
    plt.show() # shows plot of distribution after 1h of running the simulation
    return
# dictionary with input data: 
d = {
    "cRAFT": 1e-3,
    "cInitiator": 1.66e-16,
    "cMonomer": 5e-3,
    "k31": 3.6e7,
    "k32": 1.8e8,
    "k51": 3.6e11
}

runSimulation(d)